In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/BattlesStaging_12292020_WL_tagged/BattlesStaging_12292020_WL_tagged.csv"
chunk_size = 500000

text_file_reader = pd.read_csv(csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, chunksize = chunk_size, index_col=0)

In [ ]:
dfList = []
counter = 0

for df in text_file_reader:
    dfList.append(df)
    counter= counter +1
    print("Max rows read: " + str(chunk_size * counter) )



In [ ]:
df = pd.concat(dfList,sort=False)

### Load Masterlist of Cards Of Season 18

In [ ]:
cardmasterlist_csv_filename = "/kaggle/input/clash-royale-season-18-dec-0320-dataset/CardMasterListSeason18_12082020.csv"
df_cardmasterlist = pd.read_csv(cardmasterlist_csv_filename, engine='python',encoding='utf-8-sig', quoting=csv.QUOTE_MINIMAL, index_col = 0)
df_cardmasterlist.reset_index(inplace=True)
cardmasterlist_dict = dict(zip(df_cardmasterlist["team.card1.id"],df_cardmasterlist["team.card1.name"]))

### Define Arena Placer

In [ ]:
def arena_placer(row):
    arena_name = None
    
    if(row['average.startingTrophies'] < 300):
        arena_name = '01.Goblin Stadium'
    elif ((row['average.startingTrophies'] >= 300)&(row['average.startingTrophies'] < 600)):
        arena_name = '02.Bone Pit' 
    elif ((row['average.startingTrophies'] >= 600)&(row['average.startingTrophies'] < 1000)):
        arena_name = '03.Barbarian Bowl'
    #1000
    elif ((row['average.startingTrophies'] >= 1000)&(row['average.startingTrophies'] < 1300)):
        arena_name = '04.P.E.K.K.A\'s Playhouse'
    elif ((row['average.startingTrophies'] >= 1300)&(row['average.startingTrophies'] < 1600)):
        arena_name = '05.Spell Valley'  
    elif ((row['average.startingTrophies'] >= 1600)&(row['average.startingTrophies'] < 2000)):
        arena_name = '06.Builder\'s Workshop'
    # 2000
    elif ((row['average.startingTrophies'] >= 2000)&(row['average.startingTrophies'] < 2300)):
        arena_name = '07.Royal Arena'
    elif ((row['average.startingTrophies'] >= 2300)&(row['average.startingTrophies'] < 2600)):
        arena_name = '08.Frozen Peak'  
    elif ((row['average.startingTrophies'] >= 2600)&(row['average.startingTrophies'] < 3000)):
        arena_name = '09.Jungle Arena'
    # 3000
    elif ((row['average.startingTrophies'] >= 3000)&(row['average.startingTrophies'] < 3300)):
        arena_name = '10.Hog Mountain'
    elif ((row['average.startingTrophies'] >= 3300)&(row['average.startingTrophies'] < 3600)):
        arena_name = '11.Electro Valley'  
    elif ((row['average.startingTrophies'] >= 3600)&(row['average.startingTrophies'] < 4000)):
        arena_name = '12.Spooky Town'
    # 4000
    elif ((row['average.startingTrophies'] >= 4000)&(row['average.startingTrophies'] < 4300)):
        arena_name = '13.Challenger I'
    elif ((row['average.startingTrophies'] >= 4300)&(row['average.startingTrophies'] < 4600)):
        arena_name = '14.Challenger II'
    elif ((row['average.startingTrophies'] >= 4600)&(row['average.startingTrophies'] < 5000)):
        arena_name = '15.Challenger III'
    # 5000
    elif ((row['average.startingTrophies'] >= 5000)&(row['average.startingTrophies'] < 5300)):
        arena_name = '16.Master I'
    elif ((row['average.startingTrophies'] >= 5300)&(row['average.startingTrophies'] < 5600)):
        arena_name = '17.Master II'
    elif ((row['average.startingTrophies'] >= 5600)&(row['average.startingTrophies'] < 6000)):
        arena_name = '18.Master III'
    # 6000
    elif ((row['average.startingTrophies'] >= 6000)&(row['average.startingTrophies'] < 6300)):
        arena_name = '19.Champion'
    elif ((row['average.startingTrophies'] >= 6300)&(row['average.startingTrophies'] < 6600)):
        arena_name = '20.Grand Champion'
    elif ((row['average.startingTrophies'] >= 6600)&(row['average.startingTrophies'] < 7000)):
        arena_name = '21.Royal Champion'         
    elif(row['average.startingTrophies'] >= 7000):
        arena_name = '22.Ultimate Champion' 
        
    return arena_name

df['average.startingTrophies'] = df['average.startingTrophies'].fillna(0)
df['arena.name'] = None
df['arena.name'] = df.apply(arena_placer, axis=1)

### Get a subset of the Data

### Pivot The Values Doing Length Grouped By Arena Name

In [ ]:
df_winner_card1 = df[['arena.name', 'winner.card1.id']].pivot_table(columns='winner.card1.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card2 = df[['arena.name', 'winner.card2.id']].pivot_table(columns='winner.card2.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card3 = df[['arena.name', 'winner.card3.id']].pivot_table(columns='winner.card3.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card4 = df[['arena.name', 'winner.card4.id']].pivot_table(columns='winner.card4.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card5 = df[['arena.name', 'winner.card5.id']].pivot_table(columns='winner.card5.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card6 = df[['arena.name', 'winner.card6.id']].pivot_table(columns='winner.card6.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card7 = df[['arena.name', 'winner.card7.id']].pivot_table(columns='winner.card7.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_winner_card8 = df[['arena.name', 'winner.card8.id']].pivot_table(columns='winner.card8.id', index=['arena.name'], aggfunc=len, fill_value=0)


In [ ]:
df_loser_card1 = df[['arena.name', 'loser.card1.id']].pivot_table(columns='loser.card1.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card2 = df[['arena.name', 'loser.card2.id']].pivot_table(columns='loser.card2.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card3 = df[['arena.name', 'loser.card3.id']].pivot_table(columns='loser.card3.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card4 = df[['arena.name', 'loser.card4.id']].pivot_table(columns='loser.card4.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card5 = df[['arena.name', 'loser.card5.id']].pivot_table(columns='loser.card5.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card6 = df[['arena.name', 'loser.card6.id']].pivot_table(columns='loser.card6.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card7 = df[['arena.name', 'loser.card7.id']].pivot_table(columns='loser.card7.id', index=['arena.name'], aggfunc=len, fill_value=0)
df_loser_card8 = df[['arena.name', 'loser.card8.id']].pivot_table(columns='loser.card8.id', index=['arena.name'], aggfunc=len, fill_value=0)


### Combine the values using vector addition

In [ ]:
df_winner_combined = df_winner_card1 + df_winner_card2 + df_winner_card3 + df_winner_card4 + df_winner_card5 + df_winner_card6 + df_winner_card7 + df_winner_card8
df_loser_combined = df_loser_card1 + df_loser_card2 + df_loser_card3 + df_loser_card4 + df_loser_card5 + df_loser_card6 + df_loser_card7 + df_loser_card8
df_grand_totals = df_winner_combined + df_loser_combined

In [ ]:
df_grand_totals = df_grand_totals.rename(columns=cardmasterlist_dict)

In [ ]:
df_grand_totals.index = [x.split(".",1)[1] for x in df_grand_totals.index]

### Let's do an eye check

In [ ]:
df_grand_totals

### Let's Transpose columns to rows and vice-versa

In [ ]:
df_grand_totals.T

### TODO
### graphs, polishing, etc